In [1]:
from foods_and_recipes import foods_df, recipes_df, RECIPES_BY_NAME
from nutrition_models import DogProfile, weight_loss_target_kcal, weight_loss_target_kj

oakley = DogProfile(name="Oakley", current_weight_kg=40.0, ideal_weight_kg=35.0)

target_kcal = weight_loss_target_kcal(oakley.ideal_weight_kg)
target_kj = weight_loss_target_kj(oakley.ideal_weight_kg)

df_foods = foods_df()
df_recipes = recipes_df()

base = RECIPES_BY_NAME["Base daily stew"]
base.total_kj, base.total_kcal, base.total_cost, target_kj, target_kcal

(2830.0, 676.3862332695984, 4.125, 3371.5587840990784, 805.8218891250187)